In [ ]:
import  numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold, datasets

# generate two datsets
def generateHeart(num_points):

  #source
  #https://medium.com/the-stata-guide/i-stata-391736dc6bad

  #t = np.linspace(0, np.pi*2, num_points+1)[:-1]

  t = 2*np.pi * np.random.random_sample(num_points)
  x = np.sin(t)**3  #+ np.random.normal(0,0.05, num_points)
  y = 13*np.cos(t)-5*np.cos(2*t)-2*np.cos(3*t)-np.cos(4*t) #+ np.random.normal(0,0.05, num_points)

  # rescale
  y += (np.abs(np.min(y)) - np.max(y))/2
  y /= np.max(y)

  # x points, y points and colors
  return np.array([x,y,t]).T


def generateCircle(num_points):
  t = 2*np.pi * np.random.random_sample(num_points)
  #t = np.linspace(0, np.pi*2, num_points+1)[:-1]
  x = np.sin(t) #+ 2#+ np.random.normal(0,0.05, num_points)
  y = np.cos(t) #+ np.random.normal(0,0.05, num_points)

  return np.array([x,y,t]).T

def generateLine(num_points):
  x = 2 * np.random.random_sample(num_points) - 1
  #x = np.linspace(-1, 1, num_points)
  y = np.zeros(num_points) #+ np.random.normal(0,0.05, num_points)

  return np.array([x,y,x]).T

def generateSwirl(num_points):

  data =  datasets.make_swiss_roll(n_samples = num_points)
  return np.hstack((np.hstack((data[0][:,:1], data[0][:,2:])), np.reshape(data[1], (num_points,1))))

def generateCircles(num_points):
  t = 2*np.pi * np.random.random_sample(num_points)
  #t = np.linspace(0, np.pi*2, num_points+1)[:-1]
  x = np.sin(t) #+ 2#+ np.random.normal(0,0.05, num_points)
  y = np.cos(t) #+ np.random.normal(0,0.05, num_points)
  half = int(num_points/2)
  x[:half] = x[:half]*2
  y[:half] = y[:half]*2
  return np.array([x,y,t]).T


def plot(X):

  fig, ax = plt.subplots(figsize=(6, 6))
  #ax.set(xlim=(-1.3, 1.3), ylim=(-1.3, 1.3))
  ax.scatter(X[:,0],X[:,1], c=X[:,2], s=20.0)

n = 40000
heart = generateHeart(n)
circle = generateCircle(n)
line = generateLine(n)
swirl = generateSwirl(n)
circles = generateCircles(n)

h_c = heart[:,2:]
c_c = circle[:,2:]
l_c = line[:,2:]
plot(heart)
plot(circle)
plot(line)
plot(swirl)
plot(circles)
#np.savetxt("hearts.txt", hearts[:,:2].astype('float32'), header=(str(n)))
#np.savetxt("circles.txt", circles[:,:2].astype('float32'), header=(str(n)))

#X = np.genfromtxt("outhearts.txt", dtype='float')
#print(X)
#plot(np.hstack((X,hearts_c)))


In [ ]:
!pip install pykeops > install.log

import pykeops
pykeops.test_numpy_bindings()

pyKeOps with numpy bindings is working!


In [ ]:
import torch
from pykeops.torch import LazyTensor
from pykeops.torch import Vi, Vj, Pm
import time


def sinkhorn(x, y, a, b, epsf, tau, numIter, stop, numEps):

  m = np.shape(x)[0]
  n = np.shape(y)[0]
  dim = np.shape(x)[1]
  torch.set_default_dtype(torch.float64)

  use_cuda = torch.cuda.is_available()
  print("cuda:", use_cuda)

  X = torch.from_numpy(np.ascontiguousarray(x[:,None,:])).cuda()
  Y = torch.from_numpy(np.ascontiguousarray(y[None,:,:])).cuda()

  if not a:
    a = torch.full((m,1), 1.0/m).cuda()
  if not b:
    b = torch.full((n,1), 1.0/n).cuda()

  u = torch.full((m,1), 1.0, device='cuda')
  v = torch.full((n,1), 1.0, device='cuda')
  x = LazyTensor(X)
  y = LazyTensor(Y)
  alpha = torch.log(u)
  beta = torch.log(v)

  orig_u = u
  orig_v = v
  eps0 = 100
  lam = (epsf/eps0)**(1/numEps)

  for j in range(1,numEps+1):

    eps = eps0*(lam**j)
    K = (((-1*(((x - y) ** 2).sum(-1))) + Vi(alpha) + Vj(beta))/eps).exp()
    for i in range(numIter):
      v = torch.div(b, (K.T @ u))
      u = torch.div(a, (K @ v))

      logu = torch.log(u)
      logv = torch.log(v)

      alpha = torch.add(alpha, torch.mul(logu, eps))
      beta = torch.add(beta, torch.mul(logv, eps))

      u = orig_u
      v = orig_v

      if i%10 == 0:
        G = ((((-1*(((x - y) ** 2).sum(-1))) + Vi(alpha) + Vj(beta))/eps) + Vi(logu) + Vj(logv)).exp()
        err = (torch.norm(torch.sub(G.sum(0), b))).item()

        if err < stop:
          #print("early stop", i, err)
          break

      K = (((-1*(((x - y) ** 2).sum(-1))) + Vi(alpha) + Vj(beta))/eps).exp()
  G = torch.exp(torch.add(torch.div(torch.add(torch.mul(torch.sum(torch.pow(torch.sub(X,Y), 2),2),-1), torch.add(alpha, torch.transpose(beta, 0, 1))), eps), torch.add(logu, torch.transpose(logv,0,1))))

  #print("maxed out", err)
  return (G.cpu()).numpy()

G = sinkhorn(heart[:,:2], circle[:,:2], None, None, 1e-4, 0, 100, 1e-6, 10)
new_d =  G @ circle *n # heart -> cirlce mapping
plot(np.hstack((new_d[:,:2], h_c)))
